# MGRA Series 13 Specific Outputs

In [ ]:
zones = ['college', 'cpa', 'elementary', 'jurisdiction', 
         'msa', 'region', 'sdcouncil', 'secondary', 'sra', 
         'supervisorial', 'tract', 'transit', 'unified', 'zip']

final_total = pd.DataFrame(columns=['SQL', 'Extract'], index=zones)

for particular_zone in zones:
    temp_total = 0
    for i in range(8):
        zone = particular_zone

        list = [f"""with t AS

        (
        SELECT geozone, yr_id, age_group.name, SUM(population) as population 
                                FROM fact.age as tbl
                                INNER JOIN dim.mgra AS mgra
                                ON mgra.mgra_id = tbl.mgra_id
                                INNER JOIN dim.age_group
                                ON age_group.age_group_id = tbl.age_group_id
                                WHERE tbl.datasource_id IN (12,5) AND mgra.geotype = '{zone}'
                                GROUP BY mgra.geozone, yr_id, tbl.age_group_id, age_group.name
                                --ORDER BY mgra.geozone, yr_id, tbl.age_group_id
        )

        SELECT sum(population) AS total
        from t
                                """,f"""
                                with t AS

        (
        SELECT geozone, yr_id, age_group.name as age_group, sex.sex as sex, 
                                ethnicity.long_name as ethnicity, SUM(population) as population 
                                FROM fact.age_sex_ethnicity as tbl
                                INNER JOIN dim.mgra AS mgra
                                ON mgra.mgra_id = tbl.mgra_id
                                INNER JOIN dim.age_group
                                ON age_group.age_group_id = tbl.age_group_id
                                INNER JOIN dim.sex
                                ON sex.sex_id = tbl.sex_id
                                INNER JOIN dim.ethnicity
                                ON ethnicity.ethnicity_id = tbl.ethnicity_id
                                WHERE tbl.datasource_id IN (12,5) AND mgra.geotype = '{zone}'
                                GROUP BY mgra.geozone, yr_id, tbl.age_group_id, 
                                tbl.sex_id, tbl.ethnicity_id, 
                                age_group.name, sex.sex, ethnicity.long_name
                                --ORDER BY mgra.geozone, yr_id, tbl.age_group_id,
                                --tbl.sex_id, tbl.ethnicity_id
        )

        SELECT sum(population) AS total
        from t

                                """,f"""
                                with t AS

        (
        SELECT geozone, yr_id, ethnicity.long_name, SUM(population) as population 
                                FROM fact.ethnicity as tbl
                                INNER JOIN dim.mgra AS mgra
                                ON mgra.mgra_id = tbl.mgra_id
                                INNER JOIN dim.ethnicity
                                ON ethnicity.ethnicity_id = tbl.ethnicity_id
                                WHERE tbl.datasource_id IN (12,5) AND mgra.geotype = '{zone}'
                                GROUP BY mgra.geozone, yr_id, tbl.ethnicity_id, ethnicity.long_name
                                --ORDER BY mgra.geozone, yr_id, tbl.ethnicity_id
        )

        SELECT sum(population) AS total
        from t
                                """,f"""
                                with t AS

        (
        SELECT geozone, yr_id, income_group.name, SUM(households) as households 
                                FROM fact.household_income as tbl
                                INNER JOIN dim.mgra AS mgra
                                ON mgra.mgra_id = tbl.mgra_id
                                INNER JOIN dim.income_group
                                ON income_group.income_group_id = tbl.income_group_id
                                WHERE tbl.datasource_id IN (12,5) AND mgra.geotype = '{zone}'
                                GROUP BY mgra.geozone, yr_id, tbl.income_group_id, income_group.name
                                --ORDER BY mgra.geozone, yr_id, tbl.income_group_id
        )

        SELECT sum(households) AS total
        from t

                                """,f"""
                                with t AS

        (
        SELECT geozone, yr_id, structure_type.long_name, SUM(units) as units, 
                                SUM(unoccupiable) as unoccupiable, SUM(occupied) as occupied, SUM(vacancy) as vacancy
                                FROM fact.housing as tbl
                                INNER JOIN dim.mgra AS mgra
                                ON mgra.mgra_id = tbl.mgra_id
                                INNER JOIN dim.structure_type
                                ON structure_type.structure_type_id = tbl.structure_type_id
                                WHERE tbl.datasource_id IN (12,5) AND mgra.geotype = '{zone}'
                                GROUP BY mgra.geozone, yr_id, tbl.structure_type_id, structure_type.long_name
                                -- ORDER BY mgra.geozone, yr_id, tbl.structure_type_id
        )

        SELECT sum(units) + sum(unoccupiable) + sum(occupied) + sum(vacancy) AS total
        from t
                                """,f"""
                                with t AS

        (
        SELECT geozone, yr_id, employment_type.full_name, SUM(jobs) as jobs
                                FROM fact.jobs as tbl
                                INNER JOIN dim.mgra AS mgra
                                ON mgra.mgra_id = tbl.mgra_id
                                INNER JOIN dim.employment_type
                                ON employment_type.employment_type_id = tbl.employment_type_id
                                WHERE tbl.datasource_id IN (12,5) AND mgra.geotype = '{zone}'
                                GROUP BY mgra.geozone, yr_id, tbl.employment_type_id, employment_type.full_name
                                --ORDER BY mgra.geozone, yr_id, tbl.employment_type_id
        )

        SELECT sum(jobs) AS total
        from t

                                """,f"""
                                with t AS

        (
        SELECT geozone, yr_id, housing_type.long_name, SUM(population) as population
                                FROM fact.population as tbl
                                INNER JOIN dim.mgra AS mgra
                                ON mgra.mgra_id = tbl.mgra_id
                                INNER JOIN dim.housing_type
                                ON housing_type.housing_type_id = tbl.housing_type_id
                                WHERE tbl.datasource_id IN (12,5) AND mgra.geotype = '{zone}'
                                GROUP BY mgra.geozone, yr_id, tbl.housing_type_id, housing_type.long_name
                                --ORDER BY mgra.geozone, yr_id, tbl.housing_type_id
        )

        SELECT sum(population) AS total
        from t

                                """,f"""
                                with t AS

        (
        SELECT geozone, yr_id, sex.sex, SUM(population) as population
                                FROM fact.sex as tbl
                                INNER JOIN dim.mgra AS mgra
                                ON mgra.mgra_id = tbl.mgra_id
                                INNER JOIN dim.sex
                                ON sex.sex_id = tbl.sex_id
                                WHERE tbl.datasource_id IN (12,5) AND mgra.geotype = '{zone}'
                                GROUP BY mgra.geozone, yr_id, tbl.sex_id, sex.sex
                                --ORDER BY mgra.geozone, yr_id, tbl.sex_id
        )

        SELECT sum(population) AS total
        from t

                                """]

        conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                        'Server=DDAMWSQL16.sandag.org;'
                        'Database=demographic_warehouse;'
                        'Trusted_Connection=yes;')

        sql_data =  pd.read_sql_query(list[i], conn)

        temp_total = temp_total + float(sql_data['total'])
    
    print(particular_zone, temp_total)
    final_total['SQL'][particular_zone] = temp_total